In [1]:
# %pip install evaluate 

In [2]:
from transformers import (
    pipeline,                          # 파이프라인: 모델 추론을 위한 가장 쉽고 고수준의 API (전처리, 모델 실행, 후처리를 한 번에 수행)
    AutoTokenizer,                     # AutoTokenizer: 사전 학습된 모델의 이름만으로 해당 모델에 맞는 토크나이저를 자동으로 로드
    AutoModelForSeq2SeqLM,            # AutoModelForSeq2SeqLM: Seq2Seq(Sequence-to-Sequence) 작업을 위한 모델을 자동으로 로드 (예: 번역, 요약)
    T5TokenizerFast,                  # T5TokenizerFast: T5 모델 전용의 고속 토크나이저 (Rust 기반으로 구현되어 빠름)
    T5ForConditionalGeneration,       # T5ForConditionalGeneration: T5 모델의 조건부 생성(Conditional Generation) 작업을 위한 클래스
    BartForConditionalGeneration,     # BartForConditionalGeneration: BART 모델의 조건부 생성 작업을 위한 클래스
    PreTrainedTokenizerFast,          # PreTrainedTokenizerFast: 모든 고속 토크나이저의 기본 클래스
    DataCollatorForSeq2Seq,           # DataCollatorForSeq2Seq: Seq2Seq 학습 시 배치를 만들 때 패딩(padding) 등을 처리해주는 클래스
    Seq2SeqTrainingArguments,         # Seq2SeqTrainingArguments: Seq2Seq 모델 학습을 위한 하이퍼파라미터 설정 클래스
    Seq2SeqTrainer,                   # Seq2SeqTrainer: Seq2Seq 모델 학습 및 평가를 위한 트레이너 클래스
)


In [3]:
import os
import torch
import numpy as np 
from datetime import datetime

In [4]:
# 데이타셋 라이브러리
from datasets import load_dataset

In [5]:
# 버젼확인
print(f'pytorch 버젼: {torch.__version__}')
print(f'transformers 버젼: {__import__("transformers").__version__}')

pytorch 버젼: 2.5.1+cu121
transformers 버젼: 4.57.1


In [6]:
# pipeline을 이용한 간단한 문서 요약
''' 
1.모델 다운로드 및 로딩
2.토크나이제이션(문자->숫자)
3.모델추론(요약생성)
4.디코딩(숫자->문자)

장점: 적은 코드로 실행가능(2~3줄)
단점: 세밀한 제어 어려움

언제:
    빠른 프로트타입 제작
    간단한 데모
    성능테스트
'''
summarizer = pipeline(
                'summarization',    # 작업 유형: 요약(summarization)
                model = "facebook/bart-large-cnn", # 사용할 모델: Facebook에서 개발한 BART 모델 (CNN Daily Mail 데이터셋으로 학습됨)
                device = 0 if torch.cuda.is_available() else -1 # GPU 사용 설정: 0은 첫 번째 GPU, -1은 CPU 사용
                )
ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""
summary_result=summarizer(ARTICLE,
                          max_length=130,
                          min_length=30,
                          do_sample=False
                          )
print(summary_result[0]['summary_text'])



Device set to use cuda:0


Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002. She is believed to still be married to four men.


In [7]:
summary_text=summary_result[0]['summary_text']
# 통계분석
compression_ratio = len(summary_text) / len(ARTICLE)*100
word_educed = len(ARTICLE.split()) - len(summary_text.split())
print(f'원문 길이 : {len(ARTICLE):4d} 문자(약 {len(ARTICLE.split()):3d}개의 단어)')
print(f'요약 길이 : {len(summary_text):4d} 문자(약 {len(summary_text.split()):3d}개의 단어)')
print(f'압축률 : {compression_ratio:1f}%')
print(f'단어 절감 : {word_educed}개')

원문 길이 : 2298 문자(약 369개의 단어)
요약 길이 :  262 문자(약  47개의 단어)
압축률 : 11.401218%
단어 절감 : 322개


In [8]:
# T5 모델과 AutoModel을 이용한 문서 요약    Google 2019년
# 모든 nlp 작업을 텍스트 -> 텍스트 형식으로 통일
# 요약 : "summarize: [원문] -> [요약문]"
# 번역 : "translate English to French: [원문] -> [번역문]"
# 분류 : "sentiment [원문] -> [클래스]" i love this -> positive

# 단점 : Task Prefix 필수(없으면 급격한 성능 저하) -> 위의 예시처럼 문제의 유형을 알려주는
                                                        # 짧은 문장을 입력 앞에 붙임
MODEL_NAME = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

# gpu 이동(가능하면)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# 전처리
# 1. 공백 제거: 앞뒤 불필요한 공백 제거
# 2. 줄바꿈을 공백으로 변환: T5 모델은 줄바꿈 문자를 잘 처리하지 못할 수 있으므로 공백으로 대체
# 3. Task Prefix 추가 **필수**: T5는 멀티태스크 모델이므로 수행할 작업(요약, 번역 등)을 명시해야 함
preprocessed_text = ARTICLE.strip().replace("\n", " ")
input_text = f"summarize:{preprocessed_text}"

# 토크나이제이션
tokenized_text = tokenizer.encode(
    input_text,
    return_tensors="pt",
    max_length=512,
    truncation=True
).to(device)

print(f'토큰수 : {tokenized_text.shape[1]}')
print(f'텐서형태 : {tokenized_text.shape} (배치크기=1, 시퀀스길이 ={tokenized_text.shape[1]})')
print(f'첫 10개토큰 : {tokenized_text[0][:10].tolist()}')

# 디코딩(숫자 -> 텍스트)
decoded_sample = tokenizer.decode(tokenized_text[0][:20], skip_special_tokens=True)
print(f'첫 210개 디코딩 결과: {decoded_sample}')


토큰수 : 512
텐서형태 : torch.Size([1, 512]) (배치크기=1, 시퀀스길이 =512)
첫 10개토큰 : [21603, 10, 6861, 1060, 41, 254, 17235, 61, 10555, 301]
첫 210개 디코딩 결과: summarize:New York (CNN)When Liana Barrientos was 23 years old,


In [9]:
# 요약생성 Beam Search 사용
# 여러 가능성을 동시에 탐색하면서 최적의 요약 찾기
# beam=4, min_length=30, max_length=100
summary_ids = model.generate(
                tokenized_text,
                num_beams=4,            # Beam Search 크기: 4개의 가능성 높은 경로를 동시에 탐색 (값이 클수록 품질은 좋아지나 속도는 느려짐)
                no_repeat_ngram_size=3, # 3-gram 반복 방지: 동일한 3개의 단어 연속이 다시 나오지 않도록 하여 문장의 반복을 줄임
                min_length=30,          # 생성될 요약문의 최소 길이
                max_length=100,         # 생성될 요약문의 최대 길이
                early_stopping=True     # 모든 Beam이 EOS(문장 끝) 토큰을 만나면 생성을 조기 종료
                )
# 생성완료
# 디코딩
output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(f'요약 통계')
print(f'원문의 길이 : {len(ARTICLE)}문자')
print(f'요약문의 길이 :{len(output)}문자')
print(f'압축률 : {len(output) / len(ARTICLE) * 100:.1f}%')
print(f'생성 토큰수: {summary_ids.shape[1]} 개')
print(f'요약문: {output}')


요약 통계
원문의 길이 : 2298문자
요약문의 길이 :217문자
압축률 : 9.4%
생성 토큰수: 54 개
요약문: in 2010, she married once more, this time in the Bronx. she is facing two criminal counts of "offering a false instrument for filing in the first degree" prosecutors say the marriages were part of an immigration scam.


In [10]:
# T5 모델
# Task prefix
# 번역


In [11]:
# %pip install tensorflow

In [12]:
# %pip install --upgrade transformers